In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from tensorflow import keras
from kerastuner.tuners import RandomSearch

2023-10-31 13:52:25.256088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 13:52:25.382044: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-31 13:52:25.385517: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-31 13:52:25.385530: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
# Load your dataset
df = pd.read_csv('updated_Data_till_T0_2-75_durga_oct17.csv')

# Shuffle the DataFrame
data = df.sample(frac=1, random_state=42)

# Select the first 100 rows for training and testing
data = data[:10]

In [3]:
# Define input features and target variables
input_features = ['n1', 'l1', 'j1', 'LP', 'Te', 'Z', 'A', 'Rfq', 'T0' ]
target_variables = ['n2', 'l2', 'j2', 'n3', 'l3', 'j3']

In [6]:
# Split the dataset into input (X) and target (y) variables
X = data[input_features]
y = data[target_variables]
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



In [ ]:


def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh'])))
    model.add(layers.Dense(6, activation=hp.Choice('output_activation', values=['linear', 'sigmoid', 'softmax'])))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model


In [1]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=15,
    executions_per_trial=5,
    directory='ANN',
    project_name='Trial',
    loss='mean_absolute_error',
    metrics=[
        'mean_absolute_error',
        keras.metrics.MeanSquaredError(name='mean_squared_error'),
        keras.metrics.MeanAbsolutePercentageError(name='mean_absolute_percentage_error')
    ]
)

NameError: name 'RandomSearch' is not defined

In [15]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 15 Complete [00h 00m 12s]
val_mean_absolute_error: 5.916666507720947

Best val_mean_absolute_error So Far: 1.4345515966415405
Total elapsed time: 00h 00m 47s
INFO:tensorflow:Oracle triggered exit


In [7]:

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid


In [8]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            
    model.add(Dense(units = 6, kernel_initializer= 'he_uniform', activation = 'relu')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mean_absolute_error'])
    return model
    
model = KerasRegressor(build_fn=create_model, verbose=0)


layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

grid_result = grid.fit(X_train, y_train)

[grid_result.best_score_,grid_result.best_params_]


/tmp/ipykernel_229548/201496250.py:16: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, verbose=0)
2023-10-31 13:53:45.087874: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-31 13:53:45.087936: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-31 13:53:45.087977: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (HPG-01): /proc/driver/nvidia/version does not exist
2023-10-31 13:53:45.088490: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI 

[-39.96800842285156,
 {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 30,
  'layers': [45, 30, 15]}]

In [29]:
grid_result.best_score_

-40.87155952453613

In [26]:
all_results = grid_result.cv_results_

In [27]:
all_results

{'mean_fit_time': array([0.49513268, 0.56026459, 0.80130181, 0.47986779, 0.5554255 ,
        0.81198831, 0.49934931, 0.56431608, 0.66046724, 0.72831841,
        0.71584764, 0.7692071 ]),
 'std_fit_time': array([0.01333685, 0.01310889, 0.2485555 , 0.02112565, 0.01382652,
        0.25226669, 0.02529144, 0.01149305, 0.01412798, 0.1989148 ,
        0.02804995, 0.01733514]),
 'mean_score_time': array([0.12653894, 0.11671572, 0.12872987, 0.12188139, 0.11687841,
        0.13039994, 0.12313142, 0.12951717, 0.12444358, 0.15102763,
        0.15369062, 0.14148126]),
 'std_score_time': array([0.00386928, 0.00986311, 0.00425927, 0.00796818, 0.01271208,
        0.01098706, 0.00690745, 0.0054737 , 0.00553197, 0.006523  ,
        0.01195496, 0.00941802]),
 'param_activation': masked_array(data=['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid', 'sigmoid',
                    'sigmoid', 'relu', 'relu', 'relu', 'relu', 'relu',
                    'relu'],
              mask=[False, False, False, False, False, 